In [1]:
from ultralytics import YOLO
import multiprocessing

In [11]:
player_model = YOLO("models/yolo/yolov8t.pt")
court_model = YOLO("models/yolo/yolov8c.pt")

In [12]:
player_detections = player_model.track(source="data/yolo/video.mp4", project="./", conf=0.1, stream=True, show=True)
court_detections = court_model.track(source="data/yolo/video.mp4", project="./", conf=0.1, stream=True, show=True)

In [2]:
def detect(detection_list, model_path, source):
    model = YOLO(model_path)
    detections = model.track(source=source, project="./", conf=0.1, stream=True, show=True)
    for detection in detections:
         detection_list.append(detection)

In [5]:
def detect(detection_list, model_path, source):
    model = YOLO(model_path)
    detections = model.track(source=source, project="./", conf=0.1, stream=True, show=True)
    for detection in detections:
         detection_list.append(detection)
         
def track():
    thread_manager = multiprocessing.Manager()
    player_detection_list = thread_manager.list()
    court_detection_list = thread_manager.list()

    player_thread = multiprocessing.Process(target=detect, args=(player_detection_list, "models/yolo/yolov8t.pt", "data/yolo/video.mp4"))
    court_thread = multiprocessing.Process(target=detect, args=(court_detection_list, "models/yolo/yolov8c.pt", "data/yolo/video.mp4"))

    player_thread.start()
    court_thread.start()

    index = 0
    while player_thread.is_alive() or court_thread.is_alive() or player_detection_list or court_detection_list:
        if len(player_detection_list) <= index and len(court_detection_list) <= index:
            continue

        player_data = player_detection_list[index] 
        court_data = court_detection_list[index]
        index += 1
        
    player_thread.join()
    court_thread.join()


In [6]:
track()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'detect' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(